Felix Singerman - 7970742
# Transfer Learning and Tensorflow API

We will familiarize ourselves with transfer learning and with the different techniques for monitoring and controlling the training process in tensorflow.

# VGG-16

We will use the Extended Outex texture dataset from the University of Oulu and LAGIS- FRE CNRS which can be downloaded from http://lagis-vi.univ-lille1.fr/datasets/outex.html. Using an archive called Outex TC 00030.tar.gz. This archive contains colour images of 68 different texture classes. There are 10, 880 testing images and 1, 360 training images. There are two text files which provide a label for each images. We will however use the test images for training and train images for testing. We must use Keras with the tensorflow backend for this assignment. 

In [1]:
import math
import numpy as np
import os
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
import pandas as pd

Using TensorFlow backend.


We adapt the the VGG-16 network for the task. Adding fully-connected layer(s) and a softmax classifier at the output and remove some layers.

In [2]:
colnames = ['ID']

test = pd.read_csv('Outex-TC-00030/000/train.txt', names =colnames)
test[['picture','ID']] = test.pop("ID").str.split(n=1, expand=True)
test = test.iloc[1:]

train = pd.read_csv('Outex-TC-00030/000/test.txt', names =colnames)
train[['picture','ID']] = train.pop("ID").str.split(n=1, expand=True)
train = train.iloc[1:]

classes = pd.read_csv('Outex-TC-00030/000/classes.txt', names = colnames)
classes[['picture','ID','misc']] = classes.pop("ID").str.split(n=2, expand=True)
classes = classes.iloc[1:]

train = pd.merge(train,classes, on = "ID",how='inner')
train = train.drop(['ID', 'misc'], axis=1)

test = pd.merge(test,classes, on = "ID",how='inner')
test = test.drop(['ID', 'misc'], axis=1)


train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

datagen=ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
        dataframe=train,
        directory='Outex-TC-00030/images',
        target_size=(128, 128),
        x_col="picture_x", y_col="picture_y",
        batch_size=128,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_dataframe(
        dataframe = test,
        directory = 'Outex-TC-00030/images',
        target_size=(128, 128),
        x_col="picture_x", y_col="picture_y",
        batch_size=128,
        class_mode='categorical')

train_generator_small = train_datagen.flow_from_dataframe(
        dataframe=train,
        directory='Outex-TC-00030/images',
        target_size=(36, 36),
        x_col="picture_x", y_col="picture_y",
        batch_size=32,
        class_mode='categorical')

validation_generator_small = test_datagen.flow_from_dataframe(
        dataframe = test,
        directory = 'Outex-TC-00030/images',
        target_size=(36, 36),
        x_col="picture_x", y_col="picture_y",
        batch_size=32,
        class_mode='categorical')

Found 10880 images belonging to 68 classes.
Found 1360 images belonging to 68 classes.
Found 10880 images belonging to 68 classes.
Found 1360 images belonging to 68 classes.


In [3]:
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Model


vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(128,128,3))

output = vgg16_base.output

vgg16_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [4]:
vgg16_base.layers[-1]
for layer in vgg16_base.layers:
    layer.trainable = False


In [5]:
output = Flatten(name='flatten')(output)
output = Dense(4096, activation='relu')(output)
output = Dense(4096, activation='relu')(output)
output = Dense(68, activation='softmax')(output)

model = Model(vgg16_base.input, output)

model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [ ]:

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size
history_vgg16 = model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=validation_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=32
)

Epoch 1/32
85/85 [==============================] - 1062s 12s/step - loss: 2.8323 - acc: 0.3498 - val_loss: 1.0328 - val_acc: 0.6258
Epoch 2/32
85/85 [==============================] - 1039s 12s/step - loss: 0.8485 - acc: 0.6903 - val_loss: 0.7316 - val_acc: 0.7143
Epoch 3/32
85/85 [==============================] - 1047s 12s/step - loss: 0.6234 - acc: 0.7607 - val_loss: 0.5611 - val_acc: 0.7792
Epoch 4/32
85/85 [==============================] - 1075s 13s/step - loss: 0.5402 - acc: 0.7920 - val_loss: 0.4120 - val_acc: 0.8320
Epoch 5/32
85/85 [==============================] - 1077s 13s/step - loss: 0.4684 - acc: 0.8133 - val_loss: 0.5357 - val_acc: 0.7914
Epoch 6/32
85/85 [==============================] - 1080s 13s/step - loss: 0.4326 - acc: 0.8301 - val_loss: 0.5321 - val_acc: 0.8028
Epoch 7/32
85/85 [==============================] - 1075s 13s/step - loss: 0.3901 - acc: 0.8445 - val_loss: 0.3982 - val_acc: 0.8369
Epoch 8/32
16/85 [====>.........................] - ETA: 13:13 - loss

# MobileNet V2

We will use parts of the Caltech-UCSD Birds-200-2011 (CUB-200-2011) dataset. This datset contains images of 200 different species of birds and different annotations per image. We will only work with the following bird species: cardinal, american goldfinch, evening grosbeak, song sparrow, tree sparrow, house sparrow, fox sparrow, canada warbler, myrtle warbler, house wren, red-winged blackbird, boat tailed grackle. We will adapt a pre-trained network for the task, MobileNetV2.

In [2]:
from keras.applications.mobilenet_v2 import MobileNetV2

birds = ["cardinal","american_goldfinch","evening_grosbeak", "song_sparrow", "tree_sparrow", "house_sparrow",
         "fox_sparrow", "canada_warbler", "myrtle_warbler", "house_wren", "red_winged_blackbird",
         "boat_tailed_grackle"]

bounding_boxes = pd.read_csv('CUB_200_2011/CUB_200_2011/bounding_boxes.txt', sep = " ", header = None, index_col = 0)

bounding_boxes

,1,2,3,4
0,,,,
1,60.0,27.0,325.0,304.0
2,139.0,30.0,153.0,264.0
3,14.0,112.0,388.0,186.0
4,112.0,90.0,255.0,242.0
5,70.0,50.0,134.0,303.0
6,33.0,53.0,251.0,395.0
7,7.0,75.0,420.0,262.0
8,78.0,86.0,333.0,158.0
9,112.0,76.0,221.0,189.0


We begin by removing the pictures we do not need for this assignment and organizing our file structure

In [3]:
import os, glob, shutil
main = "CUB_200_2011/CUB_200_2011/"
my_dir = "CUB_200_2011/CUB_200_2011/images/"
new_dir = "CUB_200_2011/CUB_200_2011/images/goodImages"

if not os.path.isdir(main+"test"):
    os.mkdir(main+"test")

if not os.path.isdir(main+"train"):
    os.mkdir(main+"train")
    
if not os.path.isdir(new_dir):
    os.mkdir(new_dir)
    
for fname in os.listdir(my_dir):
    for bird in birds:
        if bird.lower() in fname.lower():
            shutil.move(os.path.join(my_dir, fname), os.path.join(new_dir, fname))



In [4]:
import cv2
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

img = cv2.imread(new_dir+"/047.American_Goldfinch/American_Goldfinch_0012_32338.jpg")
img2 = cv2.imread(new_dir+"/017.Cardinal/Cardinal_0025_17239.jpg")
img.shape
#img2.shape

img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)



In [5]:


fig = plt.figure()
ax1 = fig.add_subplot(2,2,1)
ax1.imshow(img)
ax2 = fig.add_subplot(2,2,2)
ax2.imshow(img2)
#img.shape
imgtest = plt.imread(new_dir+"/047.American_Goldfinch/American_Goldfinch_0012_32338.jpg")
#imgtest.shape

We will not use the given train/test split given to us since we removed alot of the images and the train/test percentage will no longer be workable. So we have a new function to split the data into test/train sets. I have set it to be 80/20

In [6]:
# helper function to split data set taken from https://github.com/keras-team/keras/issues/5862 user: daanraman 

def split_dataset_into_test_and_train_sets(all_data_dir, training_data_dir, testing_data_dir, testing_data_pct):
    # Recreate testing and training directories
    if testing_data_dir.count('/') > 1:
        shutil.rmtree(testing_data_dir, ignore_errors=False)
        os.makedirs(testing_data_dir)
        print("Successfully cleaned directory " + testing_data_dir)
    else:
        print("Refusing to delete testing data directory " + testing_data_dir + " as we prevent you from doing stupid things!")

    if training_data_dir.count('/') > 1:
        shutil.rmtree(training_data_dir, ignore_errors=False)
        os.makedirs(training_data_dir)
        print("Successfully cleaned directory " + training_data_dir)
    else:
        print("Refusing to delete testing data directory " + training_data_dir + " as we prevent you from doing stupid things!")

    num_training_files = 0
    num_testing_files = 0

    for subdir, dirs, files in os.walk(all_data_dir):
        category_name = os.path.basename(subdir)

        # Don't create a subdirectory for the root directory
        print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name

        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)

        for file in files:
            input_file = os.path.join(subdir, file)
            if np.random.rand(1) < testing_data_pct:
                shutil.copy(input_file, testing_data_dir + '/' + category_name + '/' + file)
                num_testing_files += 1
            else:
                shutil.copy(input_file, training_data_dir + '/' + category_name + '/' + file)
                num_training_files += 1

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [7]:
split_dataset_into_test_and_train_sets(new_dir, "CUB_200_2011/CUB_200_2011/train", "CUB_200_2011/CUB_200_2011/test", 0.2)

Successfully cleaned directory CUB_200_2011/CUB_200_2011/test
Successfully cleaned directory CUB_200_2011/CUB_200_2011/train
goodImages vs goodImages
047.American_Goldfinch vs goodImages
010.Red_winged_Blackbird vs goodImages
120.Fox_Sparrow vs goodImages
017.Cardinal vs goodImages
130.Tree_Sparrow vs goodImages
055.Evening_Grosbeak vs goodImages
171.Myrtle_Warbler vs goodImages
196.House_Wren vs goodImages
118.House_Sparrow vs goodImages
129.Song_Sparrow vs goodImages
162.Canada_Warbler vs goodImages
049.Boat_tailed_Grackle vs goodImages
Processed 578 training files.
Processed 138 testing files.


We create our train and validation sets using flow_from_directory as we did in assignment 2

In [8]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        "CUB_200_2011/CUB_200_2011/train",
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        "CUB_200_2011/CUB_200_2011/test",
        target_size=(128, 128),
        batch_size=32,
        class_mode='categorical')


Found 578 images belonging to 12 classes.
Found 138 images belonging to 12 classes.


Next, we import the MobileNetV2 application and apply it to our data. We create 3 new model which we will keep track of their accuracy and apply flatten to all of them before entering the next phase, applying a different regulizer to each. 

In [9]:
from keras_applications import mobilenet_v2
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Model

mobilenet_base = MobileNetV2(input_shape=(128,128,3), alpha=1.0, include_top=False, weights='imagenet',classes=12)
mobilenet_base.layers[-1]
for layer in mobilenet_base.layers:
    layer.trainable = False
output = mobilenet_base.output
output2 = mobilenet_base.output
output3 = mobilenet_base.output
mobilenet_base.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [12]:
output = Flatten(name='flatten')(output)
output2 = Flatten(name='flatten')(output2)
output3 = Flatten(name='flatten')(output3)

model1 = Model(mobilenet_base.input, output)
model2 = Model(mobilenet_base.input, output2)
model3 = Model(mobilenet_base.input, output3)

model1.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

## Regularization

Design your model for classification with three different regularizations using dropout, batch normalization and l1-regularization. Briefly compare the obtained models in terms of accuracy. 

We first use the l1-regulization following the flatten layer. Then use softmax at the output

In [10]:
from keras import regularizers
output = Dense(4096, activation='relu')(output)
output = Dense(12, input_dim=12, kernel_regularizer=regularizers.l1(0.01), 
               activity_regularizer=regularizers.l1(0.01))(output)
output = Dense(4096, activation='relu')(output)
output = Dense(12, input_dim=12, kernel_regularizer=regularizers.l1(0.01), 
               activity_regularizer=regularizers.l1(0.01))(output)
output = Dense(12, activation='softmax')(output)

model1 = Model(mobilenet_base.input, output)
model1.save('first_model')

model1.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

We then use BatchNormalization after the softmax output since historically it provides better results in practice. 

In [11]:
from keras.layers.normalization import BatchNormalization
output2 = Dense(4096, activation='relu')(output2)
output2 = Dense(4096, activation='relu')(output2)
output2 = Dense(12, activation='softmax')(output2)
output2 = BatchNormalization()(output2)

model2 = Model(mobilenet_base.input, output2)
model2.save('second_model')

model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

Finally, we apply dropout

In [12]:
#output = model3.layers[-1].output
output3 = Dense(4096, activation='relu')(output3)
output3 = Dropout(0.5)(output3)
output3 = Dense(4096, activation='relu')(output3)
output3 = Dropout(0.5)(output3)
output3 = Dense(12, activation='softmax')(output3)


model3 = Model(mobilenet_base.input, output3)
model3.save('third_model')

model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

We compile all the models using the same optimizer -- SGD. We then train them using the same parameters and will take the best one for the next step.

In [14]:
model1.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model2.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])


model3.compile(optimizer='SGD',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [82]:
model1.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=20)

Epoch 1/20
20/20 [==============================] - 30s 1s/step - loss: 56.8239 - acc: 0.1281 - val_loss: 66.5218 - val_acc: 0.3304
Epoch 2/20
20/20 [==============================] - 9s 436ms/step - loss: 55.6463 - acc: 0.1875 - val_loss: 38.3602 - val_acc: 0.2857
Epoch 3/20
20/20 [==============================] - 9s 432ms/step - loss: 50.2254 - acc: 0.2297 - val_loss: 40.2637 - val_acc: 0.2286
Epoch 4/20
20/20 [==============================] - 9s 431ms/step - loss: 47.7522 - acc: 0.2172 - val_loss: 40.3239 - val_acc: 0.1911
Epoch 5/20
20/20 [==============================] - 9s 440ms/step - loss: 43.9871 - acc: 0.2656 - val_loss: 50.8379 - val_acc: 0.3286
Epoch 6/20
20/20 [==============================] - 9s 440ms/step - loss: 41.6778 - acc: 0.3375 - val_loss: 35.0662 - val_acc: 0.2893
Epoch 7/20
20/20 [==============================] - 9s 436ms/step - loss: 35.8303 - acc: 0.4563 - val_loss: 32.7311 - val_acc: 0.4304
Epoch 8/20
20/20 [==============================] - 9s 438ms/ste

In [83]:
model2.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=10)

Epoch 1/20
20/20 [==============================] - 31s 2s/step - loss: 7.9787 - acc: 0.1250 - val_loss: 6.5308 - val_acc: 0.1071
Epoch 2/20
20/20 [==============================] - 7s 337ms/step - loss: 8.3521 - acc: 0.1359 - val_loss: 8.0368 - val_acc: 0.1250
Epoch 3/20
20/20 [==============================] - 7s 335ms/step - loss: 6.9075 - acc: 0.1344 - val_loss: 10.1355 - val_acc: 0.1286
Epoch 4/20
20/20 [==============================] - 7s 346ms/step - loss: 8.0660 - acc: 0.1469 - val_loss: 10.8227 - val_acc: 0.1107
Epoch 5/20
20/20 [==============================] - 7s 351ms/step - loss: 7.0749 - acc: 0.1656 - val_loss: 8.8910 - val_acc: 0.1464
Epoch 6/20
20/20 [==============================] - 7s 361ms/step - loss: 7.4048 - acc: 0.1469 - val_loss: 8.3666 - val_acc: 0.1679
Epoch 7/20
20/20 [==============================] - 7s 355ms/step - loss: 7.6777 - acc: 0.1812 - val_loss: 7.7169 - val_acc: 0.1571
Epoch 8/20
20/20 [==============================] - 7s 359ms/step - loss: 9.

In [84]:
model3.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=10)

Epoch 1/20
20/20 [==============================] - 31s 2s/step - loss: 4.0678 - acc: 0.3297 - val_loss: 2.5249 - val_acc: 0.4500
Epoch 2/20
20/20 [==============================] - 7s 339ms/step - loss: 1.8627 - acc: 0.6219 - val_loss: 1.5821 - val_acc: 0.5821
Epoch 3/20
20/20 [==============================] - 7s 340ms/step - loss: 1.0249 - acc: 0.7516 - val_loss: 2.1721 - val_acc: 0.5536
Epoch 4/20
20/20 [==============================] - 7s 342ms/step - loss: 0.8453 - acc: 0.8078 - val_loss: 2.3200 - val_acc: 0.5679
Epoch 5/20
20/20 [==============================] - 7s 352ms/step - loss: 0.5717 - acc: 0.8469 - val_loss: 1.5049 - val_acc: 0.6571
Epoch 6/20
20/20 [==============================] - 7s 357ms/step - loss: 0.6236 - acc: 0.8438 - val_loss: 1.7463 - val_acc: 0.6607
Epoch 7/20
20/20 [==============================] - 7s 358ms/step - loss: 0.4076 - acc: 0.8828 - val_loss: 1.3719 - val_acc: 0.7036
Epoch 8/20
20/20 [==============================] - 7s 360ms/step - loss: 0.48

#### Optimizers

Pick the best performing model from Section 3.1 and evaluate at least three different optimizers. One optimizer must be stochastic gradient descent (SGD). Briefly compare the optimizer performance

Our best model from the previous section was model3 using the dropout layer at the end. We have already trained that model with SGD as seen above and achieved a train acc of 0.9563. We will now train it with 2 more Optimizers. 

In [15]:
from keras.models import load_model

good_model=load_model('third_model')
good_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

/home/flsingerman/anaconda3/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


We will use adamax

In [89]:
good_model.compile(optimizer='adamax',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [90]:
good_model.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=10)

Epoch 1/20
20/20 [==============================] - 36s 2s/step - loss: 4.2563 - acc: 0.3656 - val_loss: 2.9089 - val_acc: 0.4607
Epoch 2/20
20/20 [==============================] - 7s 336ms/step - loss: 2.0740 - acc: 0.6250 - val_loss: 1.8084 - val_acc: 0.5679
Epoch 3/20
20/20 [==============================] - 7s 331ms/step - loss: 0.9486 - acc: 0.7406 - val_loss: 1.3474 - val_acc: 0.6607
Epoch 4/20
20/20 [==============================] - 7s 347ms/step - loss: 0.7751 - acc: 0.7922 - val_loss: 1.9501 - val_acc: 0.6036
Epoch 5/20
20/20 [==============================] - 7s 367ms/step - loss: 0.4967 - acc: 0.8422 - val_loss: 1.6232 - val_acc: 0.6536
Epoch 6/20
20/20 [==============================] - 7s 369ms/step - loss: 0.4587 - acc: 0.8719 - val_loss: 1.5515 - val_acc: 0.6571
Epoch 7/20
20/20 [==============================] - 7s 364ms/step - loss: 0.5574 - acc: 0.8391 - val_loss: 1.8864 - val_acc: 0.6107
Epoch 8/20
20/20 [==============================] - 7s 365ms/step - loss: 0.35

We will now use adam

In [16]:
good_model2 = load_model('third_model')
good_model2.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

/home/flsingerman/anaconda3/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [92]:
good_model2.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=10)

Epoch 1/20
20/20 [==============================] - 40s 2s/step - loss: 3.1411 - acc: 0.3672 - val_loss: 1.7300 - val_acc: 0.5571
Epoch 2/20
20/20 [==============================] - 7s 338ms/step - loss: 1.3363 - acc: 0.6813 - val_loss: 1.3428 - val_acc: 0.6607
Epoch 3/20
20/20 [==============================] - 7s 337ms/step - loss: 0.7084 - acc: 0.8188 - val_loss: 1.6994 - val_acc: 0.6357
Epoch 4/20
20/20 [==============================] - 7s 350ms/step - loss: 0.5035 - acc: 0.8625 - val_loss: 1.5291 - val_acc: 0.6714
Epoch 5/20
20/20 [==============================] - 7s 361ms/step - loss: 0.6007 - acc: 0.8516 - val_loss: 1.7891 - val_acc: 0.6500
Epoch 6/20
20/20 [==============================] - 7s 360ms/step - loss: 0.3742 - acc: 0.8984 - val_loss: 1.7255 - val_acc: 0.7000
Epoch 7/20
20/20 [==============================] - 7s 358ms/step - loss: 0.3676 - acc: 0.9016 - val_loss: 1.7708 - val_acc: 0.6321
Epoch 8/20
20/20 [==============================] - 7s 365ms/step - loss: 0.38

The best optimizer is is a close race with SGD and adam coming out on top of adamax with a near ~95% training accuracy

#### Multi-task Learning

Revise your model from Section 3.2 to add a regression task to the output. You will have to decide on which layers to share between the two tasks. Discuss your observations. I couldnt solve this last part. I tried several different methods with no luck. https://keras.io/getting-started/functional-api-guide/#multi-input-and-multi-output-models. I tried following the lecture where we learnt multi-task learning and tried sharing layers but couldnt solve it. 

In [17]:
# taken from https://stackoverflow.com/questions/41749398/using-keras-imagedatagenerator-in-a-regression-model user: Tathagato Rai Dastidar
def regression_flow_from_directory(flow_from_directory_gen,
            list_of_values):
    for x, y in flow_from_directory_gen:
        values = [list_of_values[y[i]] for i in range(len(y))]
        yield x, values

In [19]:
from glob import glob


class_names = glob(main+"test/*") 
class_names = sorted(class_names)
name_id_map = dict(zip(class_names, range(len(class_names))))
print(name_id_map)

label_map = (train_generator.class_indices)
print(label_map)

{'CUB_200_2011/CUB_200_2011/test/118.House_Sparrow': 5, 'CUB_200_2011/CUB_200_2011/test/120.Fox_Sparrow': 6, 'CUB_200_2011/CUB_200_2011/test/049.Boat_tailed_Grackle': 3, 'CUB_200_2011/CUB_200_2011/test/162.Canada_Warbler': 9, 'CUB_200_2011/CUB_200_2011/test/055.Evening_Grosbeak': 4, 'CUB_200_2011/CUB_200_2011/test/130.Tree_Sparrow': 8, 'CUB_200_2011/CUB_200_2011/test/010.Red_winged_Blackbird': 0, 'CUB_200_2011/CUB_200_2011/test/017.Cardinal': 1, 'CUB_200_2011/CUB_200_2011/test/171.Myrtle_Warbler': 10, 'CUB_200_2011/CUB_200_2011/test/196.House_Wren': 11, 'CUB_200_2011/CUB_200_2011/test/129.Song_Sparrow': 7, 'CUB_200_2011/CUB_200_2011/test/047.American_Goldfinch': 2}
{'196.House_Wren': 11, '055.Evening_Grosbeak': 4, '162.Canada_Warbler': 9, '049.Boat_tailed_Grackle': 3, '047.American_Goldfinch': 2, '010.Red_winged_Blackbird': 0, '129.Song_Sparrow': 7, '120.Fox_Sparrow': 6, '017.Cardinal': 1, '130.Tree_Sparrow': 8, '118.House_Sparrow': 5, '171.Myrtle_Warbler': 10}


In [45]:
# reg_train_generator = train_datagen.flow_from_directory(
#         "CUB_200_2011/CUB_200_2011/train",
#         y_col = label_map,
#         target_size=(128, 128),
#         batch_size=32,
#         class_mode='sparse')

# reg_validation_generator = test_datagen.flow_from_directory(
#         "CUB_200_2011/CUB_200_2011/test",
#         target_size=(128, 128),
#         batch_size=32,
#         class_mode='sparse')

In [21]:
# regression_train_generator = regression_flow_from_directory(
#         reg_train_generator,
#         label_map)
# print(type(reg_train_generator))
# regression_validation_generator = regression_flow_from_directory(
#         reg_validation_generator,
#         label_map)

<class 'keras_preprocessing.image.DirectoryIterator'>


We will share all the layers except for the output layer. We will then add new layers to the end inorder to make it a regression task. All the inputs are the same except for the last layer and we reduce the output to just 1 value. 

In [15]:
from keras.models import load_model
regression_model = load_model('third_model')
regression_model = Model(regression_model.inputs, regression_model.layers[-2].output) 
regression_model.summary()


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

/home/flsingerman/anaconda3/lib/python3.5/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [40]:

for layer in regression_model.layers:
    layer.trainable = False
print(regression_model.layers[-1])
print(bounding_boxes.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11788 entries, 1 to 11788
Data columns (total 4 columns):
1    11788 non-null float64
2    11788 non-null float64
3    11788 non-null float64
4    11788 non-null float64
dtypes: float64(4)
memory usage: 460.5 KB
None


In [62]:
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model, Sequential


# reg_output = regression_model.output

# reg_output = Dense(1026, activation='relu')(reg_output)
# reg_output = Dense(1026, activation='relu')(reg_output)


# Now let's get a tensor with the output of our vision model:
image_input = Input(shape=(128, 128, 3))
encoded_image = regression_model(image_input)


box_input = Input(shape=(11788,4), dtype='int32')
embedded_box = Embedding(input_dim=4, output_dim=12, input_length=(11788,4) )(box_input)
encoded_box = regression_model(4)(embedded_box)

# Let's concatenate the box vector and the image vector:
import keras
merged = keras.layers.concatenate([encoded_box, encoded_image])


output = Dense(12, activation='softmax')(merged)

# This is our final model:
vqa_model = Model(inputs=[image_input, box_input], outputs=output)

# The next stage would be training this model on actual data.

ValueError: Layer model_4 was called with an input that isn't a symbolic tensor. Received type: <class 'int'>. Full input: [4]. All inputs to the layer should be tensors.

In [55]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

vqa_model.fit_generator(
        [train_generator,bounding_boxes],
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=10)


NameError: name 'model' is not defined

In [56]:

# from keras.layers import Input, Embedding, LSTM, Dense
# from keras.models import Model

# # Headline input: meant to receive sequences of 100 integers, between 1 and 10000.
# # Note that we can name any layer by passing it a "name" argument.
# main_input = Input(shape=(128,128,3), dtype='int32', name='main_input')

# # This embedding layer will encode the input sequence
# # into a sequence of dense 512-dimensional vectors.
# x = Embedding(output_dim=512, input_dim=10000)(main_input)

# # A LSTM will transform the vector sequence into a single vector,
# # containing information about the entire sequence
# reg_out = regression_model(32)(x)

# auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

# auxiliary_input = Input(shape=(5,), name='aux_input')
# x = keras.layers.concatenate([lstm_out, auxiliary_input])

# # We stack a deep densely-connected network on top
# x = Dense(64, activation='relu')(x)
# x = Dense(64, activation='relu')(x)
# x = Dense(64, activation='relu')(x)

# # And finally we add the main logistic regression layer
# main_output = Dense(1, activation='sigmoid', name='main_output')(x)

# reg_model = Model(inputs=[main_input, auxiliary_input], outputs=[main_output, auxiliary_output])


In [57]:

# # This layer can take as input a matrix
# # and will return a vector of size 64
# shared = regression_model(64)

# # When we reuse the same layer instance
# # multiple times, the weights of the layer
# # are also being reused
# # (it is effectively *the same* layer)
# encoded_box = shared(bounding_boxes)
# encoded_image = shared(train_generator)

# # We can then concatenate the two vectors:
# merged_vector = keras.layers.concatenate([encoded_a, encoded_b], axis=-1)

# # And add a logistic regression on top
# predictions = Dense(1, activation='sigmoid')(merged_vector)

# # We define a trainable model linking the
# # tweet inputs to the predictions
# regression_model = Model(inputs=[bounding_boxes, train_generator], outputs=predictions)

# regression_model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])
# # model.fit([data_a, data_b], labels, epochs=10)

# # reg_output = regression_model.output

# # reg_output = Dense(1026, activation='relu')(reg_output)
# # reg_output = Dense(1026, activation='relu')(reg_output)

# # reg_output = Dense(12, activation='relu')(reg_output)
# # reg_output = Dense(1, kernel_initializer='normal', activation='sigmoid')(reg_output)

# # regression_model_final = Model(regression_model.input, reg_output)
# regression_model_final.save('regression_model')

In [58]:
regression_model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

In [24]:
regression_model_final.compile(optimizer='adam',
              loss=multitask_loss,
              metrics=['accuracy'])

In [80]:
regression_model_final.fit_generator(
        train_generator,
        steps_per_epoch=20,
        epochs=20,
        validation_data=validation_generator,
        validation_steps=10)

Epoch 1/20


ValueError: Error when checking target: expected dense_31 to have shape (1,) but got array with shape (12,)